# Sumário

1.  Introdução ao RAG (Retrieval-Augmented Generation)
    * Ingestão de Dados
        * Carregamento (Loading)
        * Divisão (Splitting)
        * Vetorização (Embedding)
        * Armazenamento (Storing)
        * Persistindo e Carregando o Banco de Dados Vetorial
    * Recuperação e Geração
2.  Ferramentas (Tools)
    * Definindo Ferramentas Customizadas
    * Conectando Ferramentas a um LLM
    * Executando as Ferramentas e Gerando a Resposta Final

In [ ]:
# pip install langchain langchain-openai langchain-community chromadb langchainhub python-dotenv beautifulsoup4

In [ ]:
import os
from langchain_openai import ChatOpenAI

# É uma boa prática armazenar a chave de API como uma variável de ambiente.
# os.environ["OPENAI_API_KEY"] = "SUA_CHAVE_API_AQUI"

# Instanciando o modelo de chat da OpenAI.
# O parâmetro "temperature" controla a aleatoriedade da saída.
llm = ChatOpenAI(model="gpt-4o-mini")

# 1. Retrieval-Augmented Generation (RAG)

Retrieval-Augmented Generation (RAG) é uma arquitetura que aprimora a capacidade de Modelos de Linguagem Grandes (LLMs) ao integrá-los com sistemas de recuperação de informação. Em vez de depender exclusivamente do conhecimento paramétrico internalizado durante o treinamento, o RAG permite que o modelo acesse e utilize informações de uma base de dados externa em tempo real para fundamentar suas respostas.

O processo consiste em duas fases principais:

1.  **Recuperação (Retrieval)**: Dado uma consulta (prompt) do usuário, o sistema busca em um grande corpus de documentos (por exemplo, um banco de dados vetorial) os trechos de texto mais relevantes para a consulta.
2.  **Geração (Generation)**: O LLM recebe a consulta original juntamente com os documentos recuperados e utiliza essas informações como contexto para gerar uma resposta mais precisa, detalhada e factual.

## Ingestão de Dados

A primeira etapa para construir um sistema RAG é o processamento e armazenamento da base de conhecimento que será utilizada. Este processo, conhecido como ingestão, compreende um pipeline de quatro passos: carregamento, divisão, vetorização e armazenamento.

### Carregamento (Loading)

A fase de carregamento consiste em importar os dados de suas fontes originais. Os documentos podem vir de diversos formatos, como arquivos de texto, PDFs, páginas da web, entre outros. A biblioteca LangChain oferece uma vasta gama de `DocumentLoaders` para facilitar essa tarefa.

Neste exemplo, utilizaremos o `WebBaseLoader` para carregar o conteúdo diretamente de uma página da internet.

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# URL da página a ser carregada
url = "https://portal.imd.ufrn.br/portal/institucional/historico"

# Instancia o loader com a URL especificada
loader = WebBaseLoader(
    url,
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("texto-conteudo")
        )
    )
)

# Carrega o conteúdo da página
documents = loader.load()

# Imprime uma prévia do conteúdo carregado
print(documents[0].page_content.strip())

### Divisão (Splitting)

LLMs possuem uma janela de contexto limitada, ou seja, um número máximo de tokens que podem processar de uma só vez. Documentos longos, como o conteúdo de uma página web, precisam ser divididos em pedaços menores (chunks) para que possam ser processados pelo modelo de embedding e, posteriormente, pelo LLM.

A estratégia de divisão é crucial para a eficácia do RAG. Uma divisão inadequada pode separar contextos semanticamente coesos, prejudicando a qualidade da recuperação. O `RecursiveCharacterTextSplitter` é uma abordagem robusta que tenta manter parágrafos, sentenças e palavras juntos, recursivamente dividindo o texto por uma lista de separadores.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Inicializa o divisor de texto
example_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,  # O tamanho máximo de cada chunk em caracteres
    chunk_overlap=10   # O número de caracteres de sobreposição entre chunks adjacentes
)

text = "Olá, este é um exemplo de texto que será dividido em chunks menores usando o RecursiveCharacterTextSplitter da biblioteca LangChain. O objetivo é garantir que cada chunk não exceda o tamanho máximo especificado, enquanto mantém uma sobreposição entre os chunks para preservar o contexto. Isso é especialmente útil quando se trabalha com modelos de linguagem que têm limites de tokens ou caracteres por entrada."

example_text_splitter.split_text(text)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Inicializa o divisor de texto
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # O tamanho máximo de cada chunk em caracteres
    chunk_overlap=50   # O número de caracteres de sobreposição entre chunks adjacentes
)

# Divide os documentos em chunks
chunks = text_splitter.split_documents(documents)

print(f"Número de documentos original: {len(documents)}")
print(f"Número de chunks após a divisão: {len(chunks)}")

### Vetorização (Embedding)

Para que o sistema possa realizar buscas por similaridade semântica, os chunks de texto precisam ser convertidos em representações numéricas, conhecidas como embeddings. Um modelo de embedding é uma rede neural que mapeia um texto para um vetor de alta dimensão. Textos com significados semelhantes resultarão em vetores próximos no espaço vetorial.

A distância entre dois vetores nesse espaço pode ser calculada por métricas como a similaridade de cosseno ou a distância Euclidiana.

$$\text{cosine\_similarity}(A, B) = \frac{A \cdot B}{\|A\| \|B\|} = \frac{\sum_{i=1}^{n} A_i B_i}{\sqrt{\sum_{i=1}^{n} A_i^2} \sqrt{\sum_{i=1}^{n} B_i^2}}$$

In [ ]:
from langchain_openai import OpenAIEmbeddings

# Inicializa o modelo de embedding
embedding_model = OpenAIEmbeddings()

# Exemplo de como vetorizar um único texto
example_embedding = embedding_model.embed_query("Quando o IMD foi criado?")

print(f"Dimensão do vetor de embedding: {len(example_embedding)}")
print(f"Primeiros 5 valores do embedding: {example_embedding[:5]}")

### Armazenamento (Storing)

Após a conversão dos chunks para vetores, eles são armazenados em um banco de dados especializado, chamado de *Vector Store* (ou banco de dados vetorial). Esses bancos de dados são otimizados para realizar buscas de similaridade de vetores de forma extremamente eficiente.

Neste exemplo, usaremos o ChromaDB, um banco de dados vetorial de código aberto que pode ser executado em memória, facilitando a prototipação.

In [ ]:
from langchain.vectorstores import Chroma

# Define o diretório onde o banco de dados vetorial será salvo
persist_directory = 'vecdb_imd'

# Cria o vectorstore a partir dos chunks, utilizando o modelo de embedding e especificando o diretório de persistência.
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory=persist_directory
)

### Persistindo e Carregando o Banco de Dados Vetorial

O processo de ingestão de dados (carregamento, divisão e, principalmente, vetorização) pode ser computacionalmente caro e demorado, especialmente com grandes volumes de documentos. A persistência em disco é crucial.

Uma vez que o `vectorstore` foi criado com um `persist_directory`, os dados já estão salvos. Em uma execução futura do código, podemos pular os passos de `load`, `split` e `from_documents` e carregar diretamente o banco de dados já existente.

In [ ]:
# É necessário fornecer a mesma função de embedding para que o Chroma saiba como vetorizar as buscas futuras
vectorstore = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding_model
)

## Recuperação e Geração

Com o `vectorstore` criado (ou carregado do disco), podemos criar um "retriever". O retriever é um componente que, dada uma consulta, busca os documentos mais relevantes no `vectorstore`. Em seguida, construímos a cadeia RAG completa usando a LangChain Expression Language (LCEL).

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

In [ ]:
# 1. Cria um retriever a partir do vectorstore
retriever = vectorstore.as_retriever(
    search_type="similarity",  # Tipo de busca (similaridade exata)
    search_kwargs={"k": 2}     # Número de documentos a serem retornados
)

In [ ]:
retriever.invoke("quando o imd começou com o bti?")

In [ ]:
# 2. Define o template do prompt que será enviado ao LLM
template = """
Responda a pergunta baseando-se somente no seguinte contexto:
{context}

Pergunta: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
# 3. Inicializa o LLM
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
# 4. Cria a cadeia RAG utilizando LCEL (LangChain Expression Language)

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# 5. Invoca a cadeia com uma pergunta sobre o conteúdo carregado
query = "o que aconteceu em 2015?"
response = rag_chain.invoke(query)

print(response)

# 2. Ferramentas (Tools)

O conhecimento de um LLM é estático, limitado aos dados com os quais foi treinado. Ele não sabe a data atual, não pode acessar a internet em tempo real e não consegue interagir com APIs externas. As **Ferramentas (Tools)** são a solução para essa limitação.

Uma ferramenta é, essencialmente, uma função que o LLM pode aprender a chamar. Ao disponibilizar um conjunto de ferramentas para um LLM, nós o capacitamos a:
* Obter informações atualizadas (ex: notícias, previsão do tempo).
* Executar cálculos complexos.
* Interagir com qualquer sistema externo que possua uma API.

O processo, conhecido como **Tool Calling** (ou *Function Calling*), funciona da seguinte forma: o LLM recebe o prompt do usuário e, em vez de gerar uma resposta textual, ele pode gerar uma instrução estruturada (JSON) para chamar uma ou mais ferramentas com os argumentos apropriados.

## Definindo Ferramentas

Podemos criar ferramentas a partir de qualquer função Python. O mais importante é fornecer uma *docstring* clara e descritiva para a função. O LLM utilizará essa documentação para entender o que a ferramenta faz, quais são seus parâmetros e quando ela deve ser utilizada.

In [ ]:
from langchain.tools import tool
from datetime import date

@tool
def get_current_date() -> str:
    """Retorna a data de hoje no formato AAAA-MM-DD."""
    return str(date.today())

@tool
def multiply(a: float, b: float) -> float:
    """Multiplica dois números, a e b."""
    return a * b

# Lista de ferramentas que serão disponibilizadas para o LLM
tools = [get_current_date, multiply]

## Conectando Ferramentas a um LLM

Uma vez definidas, as ferramentas precisam ser "apresentadas" ao LLM. Em LangChain, fazemos isso utilizando o método `.bind_tools()`. Esse método vincula as definições das ferramentas ao modelo, permitindo que ele as considere como possíveis ações ao processar um prompt.

Quando invocamos o modelo com as ferramentas, a resposta não será uma string, mas sim uma mensagem especial (AIMessage) que pode conter `tool_calls`. Essas são as "recomendações" do LLM sobre quais funções executar para satisfazer o pedido do usuário.

In [ ]:
from langchain_core.messages import HumanMessage

# Vinculamos as ferramentas ao modelo
llm_with_tools = llm.bind_tools(tools)

# Criamos uma lista de mensagens para a conversa
# O prompt do usuário é a última mensagem
messages = [
    HumanMessage(content="Qual a data de hoje? E quanto é 12.3 multiplicado por 4.56?")
]

# Invocamos o modelo
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

# A resposta do LLM contém as chamadas de ferramenta que ele decidiu fazer
print(ai_msg.tool_calls)

## Executando as Ferramentas e Gerando a Resposta Final

A saída do LLM no passo anterior é apenas uma instrução. Ela não executa as funções. Nosso código é responsável por:
1.  Interpretar as `tool_calls` geradas pelo modelo.
2.  Executar as funções Python correspondentes com os argumentos fornecidos.
3.  Coletar os resultados de cada chamada.
4.  Enviar esses resultados de volta para o LLM.

Ao receber os resultados, o LLM pode finalmente sintetizar uma resposta final em linguagem natural para o usuário.

In [ ]:
from langchain_core.messages import ToolMessage

# Dicionário para mapear nomes de ferramentas às suas funções
tool_map = {tool.name: tool for tool in tools}

# Iteramos sobre as chamadas de ferramenta recomendadas pelo LLM
for tool_call in ai_msg.tool_calls:
    # Obtemos a função a partir do seu nome
    tool_to_call = tool_map[tool_call["name"]]
    # Invocamos a função com os argumentos fornecidos pelo LLM
    observation = tool_to_call.invoke(tool_call["args"])
    # Adicionamos o resultado da execução da ferramenta ao histórico
    messages.append(ToolMessage(content=str(observation), tool_call_id=tool_call["id"]))

# Agora, o histórico contém a pergunta do usuário, a decisão do LLM de usar as ferramentas,
# e os resultados dessas ferramentas.
# Invocamos o modelo novamente com o histórico completo.
final_response = llm_with_tools.invoke(messages)

print(final_response.content)

### Simplificando o Ciclo de Execução de Ferramentas com LangGraph

No processo manual que vimos, nós somos responsáveis por:
1.  Receber as `tool_calls` do LLM.
2.  Executar as funções correspondentes.
3.  Formatar os resultados em `ToolMessage`.
4.  Enviar tudo de volta ao LLM para a síntese final.

Embora esse processo seja fundamental para o entendimento, ele pode se tornar repetitivo. A biblioteca **LangGraph** oferece abstrações de alto nível para construir agentes e aplicações complexas. Uma dessas abstrações é o `create_react_agent`, um "grafo" pré-construído que executa exatamente esse ciclo de raciocínio e ação para nós.

Ele gerencia o estado da conversa (o histórico de mensagens), chama as ferramentas quando necessário e alimenta os resultados de volta ao modelo até que a tarefa seja concluída e uma resposta final seja gerada. Com isso, todo o nosso loop manual é substituído por uma única chamada `.invoke()`.

In [ ]:
from langgraph.prebuilt import create_react_agent

# 1. Criamos o executor do agente com o modelo e as ferramentas
agent = create_react_agent(
    model=llm,  
    tools=tools
)

In [ ]:
# A estrutura espera uma chave "messages" contendo a lista
response = agent.invoke({
    "messages": [
        HumanMessage(content="Qual a data de hoje? E quanto é 12.3 multiplicado por 4.56?")
    ]
})

In [ ]:
# 3. O resultado contém o histórico completo da execução, incluindo as chamadas de ferramenta e a resposta final do LLM.
print("--- Histórico Completo da Execução ---")
for message in response["messages"]:
    # O atributo .pretty_print() oferece uma visualização mais clara
    message.pretty_print()

In [ ]:
# A última mensagem do assistente é a resposta consolidada
final_message = response["messages"][-1]
print(final_message.content)

# Exercícios

## Exercício 1: Construindo seu Próprio Sistema RAG

Implementar um pipeline RAG do início ao fim, utilizando um documento de sua escolha.

## Exercício 2: Criando Ferramentas de Leitura e Escrita de Arquivos

Implementar duas ferramentas para interagir com o sistema de arquivos local.